# YUPST - Bulding a Healthy Daily Meal Plan

### Imports

In [18]:
# !pip install -q openai
import openai
import os

### OpenAI Authentication
To use this notebook as shown, you must acquire an OpenAI API Key and set it as an environment variable or load it from a file.

1. Sign up at: https://platform.openai.com/signup
2. Generate a new API Key

In [19]:
# Use ONE of the following ways to authenticate to OpenAI

# 1. Use an enviroment variable (do this once per computer)
os.environ['OPENAI_API_KEY'] = 'xyz'
openai.api_key = os.getenv('OPENAI_API_KEY')


# 2. Use getpass and prompt the user for the API Key 
# import getpass
# api_key = getpass.getpass('Paste your API Key:')
# assert api_key.startswith('sk-'), 'Error loding the API key. OpenAI API Keys start with "sk-".'
# openai.api_key = api_key

# 3. Load the key from a file
#with open('key.txt', 'r') as f:
#    api_key = f.read().strip('\n')
#    assert api_key.startswith('sk-'), 'Error loading the API key. The API key starts with "sk-"' 
#openai.api_key = api_key

### Generate the meal plan based on a list of ingredients 

In [20]:
def create_meals(ingredients, kcal=2000):
    prompt = f'''Create a healthy daily meal plan for breakfast, lunch and dinner based on
    the following ingredients {ingredients}.
    Explain each recipe.
    The total daily intake of kcal should be below {kcal}.
    Assign a suggestive and concise title to each meal.
    Your answer should end with 'Titles: ' and the title of each recipe.'''
    
    #     print(prompt)
    messages = [
        {'role': 'system', 'content': 'You are a talented cook.'},
        {'role': 'user', 'content': prompt}
    ]
    
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        temperature=1,
        max_tokens = 1024,
        n=1
    )
    return response['choices'][0].message.content

foods = 'broccoli, chicken, fish, vegetables, cabbage, eggs, olive oil'
output = create_meals(foods)
print(output)

Breakfast: 
Broccoli and Egg Scramble
- In a non-stick pan, heat 1 tbsp of olive oil and sauté broccoli florets until soft. In a bowl, whisk 2 eggs and pour into the pan. Stir until cooked. Season with salt and pepper.
- 321 kcal 
Title: Broccoli and Egg Scramble

Lunch: 
Chicken and Vegetable Stir Fry 
- Cut 100g of chicken breast into small pieces and marinate with 1 tbsp of soy sauce, 1 tsp of cornstarch, and 1 tsp of olive oil. In a wok, heat 1 tbsp of olive oil and stir fry sliced vegetables (carrots, bell peppers, cabbage). Add the marinated chicken and stir until cooked through. Season with additional soy sauce if needed. 
- 411 kcal 
Title: Chicken and Vegetable Stir Fry 

Dinner: 
Baked Fish with Roasted Broccoli 
- Preheat the oven to 375°F. Season one piece of fish (120g) with salt and pepper. Coat lightly with 1 tsp of olive oil. Bake for 15-20 minutes until the fish is cooked through. Toss broccoli florets in 1 tsp of olive oil and roast in the oven for 10-12 minutes. Seas

In [21]:
titles = output.splitlines()[-3:]
print(titles)

['- Broccoli and Egg Scramble ', '- Chicken and Vegetable Stir Fry ', '- Baked Fish with Roasted Broccoli']


In [22]:
titles = [t.strip('- ') for t in titles]
print(titles)

['Broccoli and Egg Scramble', 'Chicken and Vegetable Stir Fry', 'Baked Fish with Roasted Broccoli']


### Generate images for meals using DALL-E

In [23]:
def create_and_save_image(title, extra=''):
    import requests
    import shutil
    
    image_prompt = f'{title}, {extra}, high quality food photography'
    response = openai.Image.create(
        prompt=image_prompt,
        n=1,
        size='1024x1024'
    )
    
    image_url = response['data'][0]['url']
    print()
    print(image_url)
    
    image_resource = requests.get(image_url, stream=True)
    print(image_resource.status_code)
    
    image_filename = f'{title}.png'
    if image_resource.status_code == 200:
        with open(image_filename, 'wb') as f:
            shutil.copyfileobj(image_resource.raw, f)
            return image_filename
    else:
        print('Error accessing the image!')
        return False
    

image_filename = create_and_save_image(titles[0], 'white background')
print(image_filename)
    
from PIL import Image
Image.open(image_filename)

for _ in range(3):
    image_filename = create_and_save_image(titles[_], 'white background')
    print(image_filename)
    


https://oaidalleapiprodscus.blob.core.windows.net/private/org-GuCouZdJjymi2GPw3lxi9MXo/user-YBK3SmOAZhMMCn4VCmIWQbjE/img-VB5VKGpEAJU39IPRfr3Ovrjg.png?st=2023-04-16T03%3A51%3A18Z&se=2023-04-16T05%3A51%3A18Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-04-16T03%3A04%3A41Z&ske=2023-04-17T03%3A04%3A41Z&sks=b&skv=2021-08-06&sig=sa%2Bbe4R%2B0f8HrzWu3Du/4AZHZk0jWbaj35npKNBSxvU%3D
200
Broccoli and Egg Scramble.png

https://oaidalleapiprodscus.blob.core.windows.net/private/org-GuCouZdJjymi2GPw3lxi9MXo/user-YBK3SmOAZhMMCn4VCmIWQbjE/img-RQ0vPDxkVWDaBWkzluC5EHos.png?st=2023-04-16T03%3A51%3A25Z&se=2023-04-16T05%3A51%3A25Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-04-16T03%3A52%3A50Z&ske=2023-04-17T03%3A52%3A50Z&sks=b&skv=2021-08-06&sig=7WevNaQR5ZgTNmkU/NKSztJ9zjR63jPTDbzWOcUvZx4%3D
200
Broccoli a